#### 1. RDD basic
reduce把任务发送到不同机器上执行. 这些机器各自持有map结果的一部分, 并在这部分上进行local reduce, 最后将结果发送给driver programme机器

In [20]:
val lines = sc.textFile("data.txt")
val lineLengths = lines.map(s => s.length)  //懒计算
val totalLength = lineLengths.reduce((a, b) => a + b)  // action操作,触发计算
lineLengths.persist()

lines = data.txt MapPartitionsRDD[12] at textFile at <console>:30
lineLengths = MapPartitionsRDD[13] at map at <console>:31
totalLength = 7


MapPartitionsRDD[13] at map at <console>:31

#### 2. 闭包

In [24]:
var counter = 0
// Wrong: Don't do this!!
lineLengths.foreach(x => {counter += x})
println("Counter value: " + counter)

Counter value: 0


counter = 0


0

1. 如上操作不会正常执行:  
 1. spark将rdd操作打散成多个`task`, 每个task由`executor`执行.   
 2. 在任务被执行前, spark讲rdd的闭包序列化后发送给每个`executor`, (此处是foreach). 闭包中的变量, 都是复制到`executor`所在机器上的, 每个`executor`操作的也是本地机器上的变量, 而不是driver programme机器上的变量. 因此, 在上述程序输出sounter时, 仍然输出的是driver机器内存里的counter(等于0), 而不是`executor`计算rdd后的输出结果
 
2. Accumulator  
像如上这种, 需要在集群中享变量进行和操作的情况, 需要使用`Accumulator`  
下面, 我们定义一个long型的Accumulator

In [30]:
val accum = sc.longAccumulator("my long accumlater")
sc.parallelize(Array(1,2,4,8)).foreach(x=>accum.add(x))
accum.value

accum = LongAccumulator(id: 302, name: Some(my long accumlater), value: 15)


15

#### 3. Printing elements of an RDD
1. 闭包中的print只在`executor`上执行, 而不会显现在driver机器上.  
 例如: `rdd.foreach(println) or rdd.map(println)`  
2. `rdd.collect().foreach(println)`会导致driver机器内存溢出. 因为`collect()`会收集所有executor产生的数据  
3. 少量数据打印可使用`rdd.take(100).foreach(println)`

#### 4. Working with Key-Value Pairs
1. 有一类专门操作键值对类型RDD的方法, 最普遍的就是`shuffle`, 比如使用key进行grouping或aggregating
2. 键值对型RDD中的key, 必须有`equals()`方法, 来匹配`hashCode()`方法

In [32]:
val lines = sc.textFile("data.txt")
val pairs = lines.map(s => (s, 1))
val counts = pairs.reduceByKey((a, b) => a + b)  //ShuffledRDD[24] at reduceByKey
counts.collect.foreach(println)

(22,1)
(11,1)
(333,1)


lines = data.txt MapPartitionsRDD[22] at textFile at <console>:34
pairs = MapPartitionsRDD[23] at map at <console>:35
counts = ShuffledRDD[24] at reduceByKey at <console>:36


ShuffledRDD[24] at reduceByKey at <console>:36

#### 5. transaction in partition example
1. `mapPartitionsWithIndex(func)`:  
 func(partitionId,iterator_for_each_partition)  
 第一个参数为index, 第二个参数为分区内数据的iterator

In [40]:
val rdd1 = sc.parallelize(List(1,2,3,4,5,6,7,8,9), 2) // 存在2个分区
val func = (index: Int, iter: Iterator[(Int)])=>{
    iter.toList.map(x => "partID:" +  index + ", val: " + x).iterator
}
rdd1.mapPartitionsWithIndex(func).collect.foreach(println)

partID:0, val: 1
partID:0, val: 2
partID:0, val: 3
partID:0, val: 4
partID:1, val: 5
partID:1, val: 6
partID:1, val: 7
partID:1, val: 8
partID:1, val: 9


rdd1 = ParallelCollectionRDD[32] at parallelize at <console>:30
func = > Iterator[String] = <function2>


<function2>

2. `aggregate(init_value)(func1,func2)`  
第一个参数是初始值  
第二个参数是2个函数:  
 1. 第一个函数:先对个个分区进行的操作  
 2. 第二个函数:对个个分区合并后的结果再进行合并,输出一个参数



In [44]:
//rdd1的分区情况上面已给出
rdd1.aggregate(5)(math.max(_, _), _ + _)

19

3. `aggregateByKey(init_value)(func1,func2)`  
 1. 将key值相同的item, 先在分区内进行local aggregate, 再在分区间进行跨分区的aggregate  
 2. reduceByKey(+)调用的都是同一个方法，只是aggregateByKey要底层一些，可以先局部再整体操作。
 3. `grupByKey`: 按照key分组, 返回(k,Sequence(v)), groupbykey相比reducebykey少了分组后进行聚合操作的函数制定, 因此,groupByKey后, 使用map指定聚合函数对序列操作
 

In [1]:
val pairRDD = sc.parallelize(List(("cat",2), ("cat", 5), ("mouse", 4),("cat", 12), ("dog", 12), ("mouse", 2))
                             , 2)
pairRDD.aggregateByKey(0)((x,y)=>math.max(x,y),
                          (x,y)=>x+y).collect()

pairRDD = ParallelCollectionRDD[0] at parallelize at <console>:27


[(dog,12), (cat,17), (mouse,6)]

In [10]:
val rdd1 = pairRDD.groupByKey
rdd1.collect.foreach(println)

(dog,CompactBuffer(12))
(cat,CompactBuffer(2, 5, 12))
(mouse,CompactBuffer(4, 2))


rdd1 = ShuffledRDD[5] at groupByKey at <console>:31


ShuffledRDD[5] at groupByKey at <console>:31

4. `countByKey`  
根据key计算key的数量

In [9]:
val rdd1 = sc.parallelize(List(("a", 1), ("b", 2), ("b", 2), ("c", 2), ("c", 1)))
rdd1.countByKey.foreach(println)
rdd1.countByValue//将("a", 1)当做一个元素，统计其出现的次数

(a,1)
(b,2)
(c,2)


rdd1 = ParallelCollectionRDD[15] at parallelize at <console>:29


Map((c,2) -> 1, (a,1) -> 1, (b,2) -> 2, (c,1) -> 1)

5. `cogroup(otherDataset)`   
 When called on datasets of type (K, V) and (K, W), returns a dataset of (K, (Iterable<V>, Iterable<W>)) tuples. 

In [12]:
var rdd1 = sc.makeRDD(Array(("A","1"),("B","2"),("C","3")),2)
var rdd2 = sc.makeRDD(Array(("A","a"),("C","c"),("D","d")),2)
var rdd3 = rdd1.cogroup(rdd2)
rdd3.collect.foreach(println)

(B,(CompactBuffer(2),CompactBuffer()))
(D,(CompactBuffer(),CompactBuffer(d)))
(A,(CompactBuffer(1),CompactBuffer(a)))
(C,(CompactBuffer(3),CompactBuffer(c)))


rdd1 = ParallelCollectionRDD[29] at makeRDD at <console>:31
rdd2 = ParallelCollectionRDD[30] at makeRDD at <console>:32
rdd3 = MapPartitionsRDD[32] at cogroup at <console>:33


MapPartitionsRDD[32] at cogroup at <console>:33

6. `cartesian`  
笛卡尔积

In [44]:
var rdd1 = sc.makeRDD(Array("a","b","c"),2)
var rdd2 = sc.makeRDD(Array(1,2),2)
rdd1.cartesian(rdd2).collect.foreach(println)

(a,1)
(a,2)
(b,1)
(c,1)
(b,2)
(c,2)


rdd1 = ParallelCollectionRDD[39] at makeRDD at <console>:30
rdd2 = ParallelCollectionRDD[40] at makeRDD at <console>:31


ParallelCollectionRDD[40] at makeRDD at <console>:31

7. `coalesce`与`repartition`   
二者均是重新对数据分区.   
`repartition`会对所有数据重新shuffle  
`coalesce`用于合并分区, 减小分区数量, 用于果过滤大数据集后的搞笑操作

#### 6. shuffle
1. shuffle是对数据进行're-distributing'操作的机制, 让数据通过分区分到不同的组
2. shuffle需要跨机器, 跨executors之间进行数据复制, 因此是一种开销极大的操作
3. 什么样的操作会引起shuffle:  
  1. `repartition` and `coalesce`
  2. `ByKey`操作, 除了`countByKey`, 例如`groupByKey` and `reduceByKey`  
  3. join操作: `cogroup` and `join`  
4. shuffle会引起大量网络IO, 而且当内存中无法存储下组织的数据, 还会引起磁盘IO来缓存数据

#### 7. RDD Persistence
1. RDD的任何一个分区丢失, 都会从源头通过rtansformation操作, 重新计算RDD
2. `presist()`可指定持久化级别, 而`cache()`使用默认级别持久化:MEMORY_ONLY(将RDD序列化后存入内存)  
3. `MEMORY_ONLY_SER`级别: 可以自己指定持久化序列方式
4. 尽量不使用写磁盘的持久化级别, 除非数据计算十分复杂或需要持久化的数据量很大, 从源头重新计算RDD往往更快  
5. 可以使用带副本的持久化级别, 如`MEMORY_ONLY_2`, 这会将数据持久化到集群中的2个不同节点.  
 spark所有持久化都是有容错机制的, 即分区丢失的话, 会重新计算该RDD分区的数据. 而使用带副本的持久化可以在1个分区副本丢失时继续执行任务, 而不必等带这个分区重新计算  
6. cache的数据将自动被spark清除出内存(使用LRU算法-least-recently-used). 可以手动使用RDD.unpersist()释放内存

#### 8. Broadcast变量
1. Broadcast变量在远程executor机器上共享只读变量(Accumulator是可读可写变量)

In [13]:
val broadcastVar = sc.broadcast(Array(1, 2, 3))
broadcastVar.value

broadcastVar = Broadcast(6)


[1, 2, 3]